# 라이블러리 불러오기

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

# 사용 사례

엔터프라이즈 데이터는 종종 SQL 데이터베이스에 저장됩니다.

LLM을 사용하면 자연어를 사용하여 SQL 데이터베이스와 상호 작용할 수 있습니다.

LangChain은 자연어 프롬프트를 기반으로 SQL 쿼리를 작성하고 실행할 수 있는 SQL 체인 및 에이전트를 제공합니다.

이들은 SQLAlchemy에서 지원하는 모든 SQL 언어(예: MySQL, PostgreSQL, Oracle SQL, Databricks, SQLite)와 호환됩니다.

다음과 같은 사용 사례를 지원합니다:

- 자연어 질문을 기반으로 실행할 쿼리 생성
- 데이터베이스 데이터를 기반으로 질문에 답변할 수 있는 챗봇 만들기
- 사용자가 분석하고자 하는 인사이트를 기반으로 사용자 지정 대시보드 구축

# 개요

LangChain은 SQL 데이터베이스와 상호 작용할 수 있는 도구를 제공합니다:

1. 자연어 사용자 질문을 기반으로 'SQL 쿼리 작성'
2. 쿼리 생성 및 실행을 위해 체인을 사용하여 'SQL 데이터베이스 쿼리'를 수행합니다.
3. 강력하고 유연한 쿼리를 위해 에이전트를 사용하여 'SQL 데이터베이스와 상호작용하기'


# SQLite
아래 예는 Chinook DB와의 SQLite 연결을 수행합니다.

* [이 파일](https://www.sqlitetutorial.net/sqlite-sample-database/)을 디렉터리에 `chinook.db`로 저장합니다.

이제 `chinook.db`가 디렉토리에 있습니다.

SQL 쿼리를 생성하고 실행하기 위해 `SQLDatabaseChain`을 생성해 보겠습니다.

In [11]:
# ! pip install langchain langchain-experimental openai

In [7]:
from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase.from_uri("sqlite:///chinook.db")
llm = OpenAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [8]:
db_chain.run("몇명의 직원이 있어?")



> Entering new SQLDatabaseChain chain...
몇명의 직원이 있어?
SQLQuery:SELECT COUNT(*) FROM employees
SQLResult: [(8,)]
Answer:8
> Finished chain.


'8'

In [12]:
db_chain.run("10개의 raw 를 랜덤으로 골라서, dataframe 형식으로 출력해줘")



> Entering new SQLDatabaseChain chain...
10개의 raw 를 랜덤으로 골라서, dataframe 형식으로 출력해줘
SQLQuery:SELECT * FROM tracks ORDER BY RANDOM() LIMIT 10
SQLResult: [(2734, 'Substitute', 221, 1, 1, 'Pete Townshend', 228022, 7409995, 0.99), (1161, "Don't Cry (Original)", 91, 2, 1, None, 284744, 4833259, 0.99), (1187, 'My World', 92, 1, 3, 'W. Axl Rose', 84532, 2764045, 0.99), (817, 'Lick It Up', 66, 1, 1, 'roger glover', 240274, 7792604, 0.99), (1480, 'Manic Depression', 120, 1, 1, 'Jimi Hendrix', 222302, 7289272, 0.99), (2013, 'On A Plain', 164, 1, 1, 'Kurt Cobain', 196440, 6390635, 0.99), (573, 'Africa Bamba', 46, 1, 1, 'I. Toure, S. Tidiane Toure, Carlos Santana & K. Perazzo', 282827, 9492487, 0.99), (2444, 'Your Mirror', 141, 1, 1, 'Mick Hucknall', 240666, 7893821, 0.99), (855, 'Fato Consumado', 69, 1, 7, None, 211565, 7018586, 0.99), (1997, 'Milk It', 163, 1, 1, 'Kurt Cobain', 225724, 7406945, 0.99)]
Answer:The result is a dataframe with 10 randomly selected rows from the tracks table.
> Finish

'The result is a dataframe with 10 randomly selected rows from the tracks table.'

# Case 1: Text-to-SQL query
**SQL** 쿼리를 작성할 체인을 만들어 보겠습니다:

In [9]:
from langchain.chains import create_sql_query_chain
from langchain.chat_models import ChatOpenAI

In [13]:
chain = create_sql_query_chain(ChatOpenAI(temperature=0), db)
response = chain.invoke({"question": "How many employees are there"})
print(response)

SELECT COUNT(*) FROM employees


- 사용자 질문을 기반으로 SQL 쿼리를 작성하고 나면 쿼리를 실행할 수 있습니다:

In [14]:
db.run(response)

'[(8,)]'

- 보시다시피 SQL 쿼리 빌더 체인은 쿼리를 **생성만**하고 **쿼리 실행**은 별도로 처리했습니다.

# Case 2: Text-to-SQL query and execution
- We can use `SQLDatabaseChain` from `langchain_experimental` to create and run SQL queries.

In [15]:
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain

llm = OpenAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [16]:
db_chain.run("직원이 몇명이나 있어?")



> Entering new SQLDatabaseChain chain...
직원이 몇명이나 있어?
SQLQuery:SELECT COUNT(*) FROM employees
SQLResult: [(8,)]
Answer:8
> Finished chain.


'8'

- 여기서 체인은 **쿼리 실행도 처리**하고 사용자 질문과 쿼리 결과를 기반으로 최종 답변을 제공합니다.
- 이 방식은 'SQL 인젝션'에 취약하기 때문에 사용 시 **주의**해야 합니다:

* 체인이 LLM에 의해 생성되고 검증되지 않은 쿼리를 실행하고 있습니다.
* 예: 레코드가 의도치 않게 생성, 수정 또는 삭제될 수 있음_.

- 이것이 바로 `SQLDatabaseChain`이 `랭체인_실험` 안에 있는 이유입니다.

## 사례 3: SQL 에이전트

- LangChain에는 `SQLDatabaseChain`보다 SQL 데이터베이스와 상호 작용하는 더 유연한 방법을 제공하는 SQL 에이전트가 있습니다.
- SQL 에이전트 사용의 주요 장점은 다음과 같습니다:
    - 데이터베이스의 스키마뿐만 아니라 데이터베이스의 콘텐츠(예: 특정 테이블 설명)를 기반으로 질문에 답변할 수 있습니다.
    - 생성된 쿼리를 실행하고 트레이스백을 포착하여 올바르게 다시 생성함으로써 오류로부터 복구할 수 있습니다.
- 에이전트를 초기화하기 위해 `create_sql_agent` 함수를 사용합니다.
- 이 에이전트에는 다음과 같은 도구가 포함된 `SQLDatabaseToolkit`이 포함되어 있습니다:
    * 쿼리 생성 및 실행
    * 쿼리 구문 확인
    * 테이블 설명 검색
    * ... 등

In [18]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

# from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType

db = SQLDatabase.from_uri("sqlite:///chinook.db")

agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0)),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

## Agent task example #1 - Running queries

In [19]:
agent_executor.run(
    "국가별 총 매출을 나열합니다. 어느 국가의 고객이 가장 많이 지출했나요?"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: albums, artists, customers, employees, genres, invoice_items, invoices, media_types, playlist_track, playlists, tracks
Thought: I should query the schema of the customers table to see what columns I can use to get the total sales by country.
Action: sql_db_schema
Action Input: customers
Observation: 
CREATE TABLE customers (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES employees ("EmployeeId")
)

/*
3 rows from customers table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	F

'USA'

## Agent task example #2 - Describing a Table

In [21]:
agent_executor.run("playlisttrack 테이블에 대해서 한국어로 설명해줄래?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: albums, artists, customers, employees, genres, invoice_items, invoices, media_types, playlist_track, playlists, tracks
Thought: I should query the schema of the playlisttrack table.
Action: sql_db_schema
Action Input: playlisttrack
Observation: Error: table_names {'playlisttrack'} not found in database
Thought: I should check the spelling of the table name.
Action: sql_db_schema
Action Input: playlist_track
Observation: 
CREATE TABLE playlist_track (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES tracks ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES playlists ("PlaylistId")
)

/*
3 rows from playlist_track table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Thought: I should query the first 10 rows from the playlist_track table.
Action: sql_db_query
Action Input: SELECT * FROM playlist_track LIMIT 1

'The playlisttrack table contains the PlaylistId and TrackId columns, with a primary key on both columns and foreign key references to the playlists and tracks tables. It also contains 3 sample rows with the PlaylistId of 1 and various TrackIds.'

# SQL 툴킷 확장하기

기본 제공되는 SQL 툴킷에는 데이터베이스 작업을 시작하는 데 필요한 도구가 포함되어 있지만, 에이전트의 기능을 확장하는 데 몇 가지 추가 도구가 유용할 수 있는 경우가 종종 있습니다. 이는 솔루션의 전반적인 성능을 개선하기 위해 솔루션에서 **도메인별 지식**을 사용하려고 할 때 특히 유용합니다.

몇 가지 예는 다음과 같습니다:

- Dynamic Few shot 예시 포함
- 열 필터로 사용할 고유명사의 철자 오류 찾기

이러한 특정 사용 사례를 처리하는 별도의 도구를 만들어 표준 SQL 도구 키트에 보완용으로 포함할 수 있습니다. 이 두 가지 사용자 정의 도구를 포함하는 방법을 살펴보겠습니다.

#### Dynamic Few shot 예제 포함

Dynamic Few shot 예제를 포함하려면 사용자의 질문과 의미적으로 유사한 예제를 검색하기 위해 벡터 데이터베이스를 처리하는 사용자 지정 **검색 도구**가 필요합니다.

몇 가지 예제가 포함된 사전을 만드는 것부터 시작하겠습니다:

In [22]:
few_shots = {
    "List all artists.": "SELECT * FROM artists;",
    "Find all albums for the artist 'AC/DC'.": "SELECT * FROM albums WHERE ArtistId = (SELECT ArtistId FROM artists WHERE Name = 'AC/DC');",
    "List all tracks in the 'Rock' genre.": "SELECT * FROM tracks WHERE GenreId = (SELECT GenreId FROM genres WHERE Name = 'Rock');",
    "Find the total duration of all tracks.": "SELECT SUM(Milliseconds) FROM tracks;",
    "List all customers from Canada.": "SELECT * FROM customers WHERE Country = 'Canada';",
    "How many tracks are there in the album with ID 5?": "SELECT COUNT(*) FROM tracks WHERE AlbumId = 5;",
    "Find the total number of invoices.": "SELECT COUNT(*) FROM invoices;",
    "List all tracks that are longer than 5 minutes.": "SELECT * FROM tracks WHERE Milliseconds > 300000;",
    "Who are the top 5 customers by total purchase?": "SELECT CustomerId, SUM(Total) AS TotalPurchase FROM invoices GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;",
    "Which albums are from the year 2000?": "SELECT * FROM albums WHERE strftime('%Y', ReleaseDate) = '2000';",
    "How many employees are there": 'SELECT COUNT(*) FROM "employee"',
}

- 그런 다음 질문 목록을 사용하여 검색기를 생성하고 대상 SQL 쿼리를 메타데이터로 할당할 수 있습니다:

In [23]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings()

few_shot_docs = [
    Document(page_content=question, metadata={"sql_query": few_shots[question]})
    for question in few_shots.keys()
]
vector_db = FAISS.from_documents(few_shot_docs, embeddings)
retriever = vector_db.as_retriever()

/home/restful3/anaconda3/envs/trading/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


- 이제 고유한 사용자 지정 도구를 만들어 'create_sql_agent' 함수에 새 도구로 추가할 수 있습니다:

In [24]:
from langchain.agents.agent_toolkits import create_retriever_tool

tool_description = """
이 도구는 유사한 예시를 이해하여 사용자 질문에 적용하는 데 도움이 됩니다.
이 도구에 입력하는 내용은 사용자 질문이어야 합니다.
"""

retriever_tool = create_retriever_tool(
    retriever, name="sql_get_similar_examples", description=tool_description
)
custom_tool_list = [retriever_tool]

이제 사용 사례를 고려하여 표준 SQL 에이전트 접미사를 조정하여 에이전트를 만들 수 있습니다. 이를 처리하는 가장 간단한 방법은 도구 설명에 포함시키는 것이지만, 이것만으로는 충분하지 않은 경우가 많으므로 생성자의 '접미사' 인수를 사용하여 에이전트 프롬프트에서 이를 지정해야 합니다.

In [25]:
from langchain.agents import AgentType, create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///chinook.db")
llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

custom_suffix = """
먼저 제가 알고 있는 비슷한 예제를 가져와야 합니다.
예제가 쿼리를 구성하기에 충분하다면 쿼리를 작성할 수 있습니다.
그렇지 않으면 데이터베이스의 테이블을 살펴보고 쿼리할 수 있는 항목을 확인할 수 있습니다.
그런 다음 가장 관련성이 높은 테이블의 스키마를 쿼리해야 합니다.
"""

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    extra_tools=custom_tool_list,
    suffix=custom_suffix,
)